In [ ]:
import sqlite3

conn = sqlite3.connect("mydb.db")
cursor = conn.cursor()

with open("3schema.sql", "r") as f:
    sql_script = f.read()

cursor.executescript(sql_script)
conn.commit()

In [ ]:
%load_ext sql
%sql sqlite:///mydb.db

In [5]:
%%sql
--Showing Data
SELECT * FROM Boxes

Running query in 'sqlite:///mydb.db'

Code,Contents,Value,Warehouse
0MN7,Rocks,180.0,3
4H8P,Rocks,250.0,1
4RT3,Scissors,190.0,4
7G3H,Rocks,200.0,1
8JN6,Papers,75.0,1
8Y6U,Papers,50.0,3
9J6F,Papers,175.0,2
LL08,Rocks,140.0,4
P0H6,Scissors,125.0,1
P2T6,Scissors,150.0,2


In [6]:
%%sql
select * from warehouses

Running query in 'sqlite:///mydb.db'

Code,Location,Capacity
1,Chicago,3
2,Chicago,4
3,New York,7
4,Los Angeles,2
5,San Francisco,8


In [7]:
%%sql
--- Select the warehouse code and the average value of the boxes in each warehouse.
select Warehouse, avg(Value) from boxes group by Warehouse
    

Running query in 'sqlite:///mydb.db'

Warehouse,avg(Value)
1,162.5
2,162.5
3,115.0
4,165.0
5,90.0


In [8]:
%%sql
-- Select the code of each box, along with the name of the city the box is located in.
select b.code, w.Location
from Boxes b
join warehouses w
on b.warehouse = w.Code


Running query in 'sqlite:///mydb.db'

Code,Location
0MN7,New York
4H8P,Chicago
4RT3,Los Angeles
7G3H,Chicago
8JN6,Chicago
8Y6U,New York
9J6F,Chicago
LL08,Los Angeles
P0H6,Chicago
P2T6,Chicago


In [10]:
%%sql
-- Select the warehouse codes, along with the number of boxes in each warehouse. 
select Warehouse, count(*) as nums
from boxes 
group by warehouse;

Running query in 'sqlite:///mydb.db'

Warehouse,nums
1,4
2,2
3,2
4,2
5,1


In [12]:
%%sql
-- Select the codes of all warehouses that are saturated 
-- (a warehouse is saturated if the number of boxes in it is larger than the warehouse's capacity).

select Code
   from Warehouses
   where Capacity <
   (
     select count(*)
       from Boxes
       where Warehouse = Warehouses.Code
   );

Running query in 'sqlite:///mydb.db'

Code
1


In [13]:
%%sql
-- Select the codes of all the boxes located in Chicago.
select Boxes.code 
from boxes join Warehouses
on boxes.warehouse = warehouses.code
where warehouses.location = 'Chicago'


Running query in 'sqlite:///mydb.db'

Code
4H8P
7G3H
8JN6
9J6F
P0H6
P2T6


In [14]:
%%sql
-- Create a new warehouse in New York with a capacity for 3 boxes
insert into Warehouses values(6,"New York",3)

Running query in 'sqlite:///mydb.db'

1 rows affected.

++
||
++
++

In [15]:
%%sql
-- Reduce the value of all boxes by 15%.
update boxes
set value = value *0.05

Running query in 'sqlite:///mydb.db'

11 rows affected.

++
||
++
++

In [16]:
%%sql
-- Remove all boxes with a value lower than $100.
delete from boxes
where value < 100


Running query in 'sqlite:///mydb.db'

11 rows affected.

++
||
++
++

In [17]:
%%sql
-- Remove all boxes from saturated warehouses.
delete from boxes
where warehouse in
(
SELECT Code
   FROM Warehouses
   WHERE Capacity <
   (
     SELECT COUNT(*)
       FROM Boxes
       WHERE Warehouse = Warehouses.Code
   )
)

Running query in 'sqlite:///mydb.db'

++
||
++
++

In [20]:
%%sql
-- Add Index for column "Warehouse" in table "boxes"
create index index_warehouse on Boxes(Warehouse)
select * from SQLITE_MASTER where type ='index'

Running query in 'sqlite:///mydb.db'

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(sqlite3.OperationalError) near "select": syntax error
[SQL: create index index_warehouse on Boxes(Warehouse)
select * from SQLITE_MASTER where type ='index']
(Background on this error at: https://sqlalche.me/e/20/e3q8)

